### Čo dokáže robiť tento Jupyter notebook? ###

Tento Jupyter notebook pracuje s databázou Partnerov verejného sektora Ministerstva spravodlivosti a s Registrom účtovných uzávierok Ministerstva financií. Vďaka tomu dokáže k **IČO** partnerom priradiť **SK_NACE** kódy. Spojením týchto dvoch databáz, teda vieme vyfiltrovať relevantné firmy v určitých sektoroch zadaním **SK_NACE** kódov (výstupom je zoznam IČO firiem, ktoré sa sú v registri partnerov verejného sektora).

Príklad využitia:
- Chceme urobiť analýzu všetkých zmlúv viažúcich sa na bezpečnostné služby na Slovensku.
- Chceme nájsť všetky zmluvy od dodávateľov elektriny, plynu, vody, spracujúcich odpad, poskytujúcich pripojenie na internet, atď.

Nasledujúci kód vybuduje databázu partnerov verejného sektora do tabuľky **DB_partners.csv**. 
Nasledujúce dve kódové časti netreba spúštať pokiaľ nechceme updatovať tabuľku **DB_partners.csv**.

In [ ]:
import json
import requests
import re
import pandas as pd

# Build DB of public partners
# Using 'Register partnerov verejneho sektora'
link = 'https://rpvs.gov.sk/OpenData/PartneriVerejnehoSektora'

page = 0
partners = 0
row_list = []

while link != '':
    resp = requests.get(link)
    content = resp.json()

    page += 1
    partners += len(content['value'])

    print('Page:',page,' Partners:',partners)

    for row in content['value']:
        row_list.append(row)

    try:
        link = content['@odata.nextLink']
    except:
        link = ''

    if (partners % 100 == 0):
        DB = pd.DataFrame(row_list)
        DB.to_csv('DB_partners.csv', sep = '|')

DB = pd.DataFrame(row_list)
DB.to_csv('DB_partners.csv', sep = '|')

K partnerom potrebujeme priradiť ešte **SK_NACE** kódy:

In [ ]:
import json
import requests
import re
import pandas as pd
from itertools import islice

DB = pd.read_csv('DB_partners.csv', sep = '|')
starting_row = 0

# If it was unsuccessful last time let's try to do it again :)
if not 'SK_NACE' in DB.columns:
    DB['SK_NACE'] = ''
else:
    # Find latest occurrence of SK_NACE
    for i in range(0,DB.shape[0]):
        if not pd.isnull(DB.at[i,'SK_NACE']):
            starting_row = i

for index, row in islice(DB.iterrows(), starting_row, None):
    print('Finding SK_NACE code for supplier',index+1,'out of',DB.shape[0])

    accounting_IDs = []
    SK_NACE_codes  = []

    # Get IDs of Accounting items
    if not pd.isnull(row['Ico']):
        if row['Ico'].isdigit():
            link = 'http://www.registeruz.sk/cruz-public/api/uctovne-jednotky?zmenene-od=2000-01-01&ico='+row['Ico']

            resp = requests.get(link)
            content = resp.json()

            if 'id' in content:
                accounting_IDs = content['id']
                last = accounting_IDs[-1]

                while content['pocetZostavajucichId']>0:
                    resp = requests.get(link+'?pokracovat-za-id='+str(last))
                    content = resp.json()
                    accounting_IDs = accounting_IDs + content['id']
                    last = accounting_IDs[-1]
            else:
                accounting_IDs = []

    for ID in accounting_IDs:
        link = 'http://www.registeruz.sk/cruz-public/api/uctovna-jednotka?id='+str(ID)

        resp = requests.get(link)
        content = resp.json()

        if 'skNace' in content:
            DB.at[index,'SK_NACE'] = content['skNace']

    if (index % 50 == 0):
        DB.to_csv('DB_partners.csv', sep = '|')

Nasledujúci kód je najpodstatnejší, zo súboru **SK_NACE_codes.txt** načíta zoznam SK_NACE kódov, ku firmám ktoré hľadáme
a vyprodukuje nám tabuľku spoločností **companies.csv** s IČO firiem, ktoré sú partneri verejného sektora a majú nejaký z SK_NACE kódov, ktorý sme poskytli
v zozname **SK_NACE_codes.txt**. Túto tabuľku potrebujeme pre Jupyter notebook, ktorý vie fitrovať zmluvy na základe IČO firiem.

In [ ]:
import os
import pandas as pd
import re

DB_partners = pd.read_csv('DB_partners.csv', sep=';', encoding='utf-8')

fo = open('SK_NACE_codes.txt','r')
lines = fo.readlines()
fo.close()

SK_NACE_codes = []
for SK_NACE_code in lines:
    SK_NACE_code = SK_NACE_code.strip()
    SK_NACE_codes.append(int(SK_NACE_code))

row_list = []
for index, row in DB_partners.iterrows():
    if row['SK_NACE'] in SK_NACE_codes:
        new_row = dict()

        new_row['ICO']     = row['Ico']
        new_row['Nazov']   = row['ObchodneMeno']
        if row['ObchodneMeno'] == '':
            new_row['Nazov'] = row['Meno'] + ' ' + row['Priezvisko']

        new_row['SK_NACE'] = row['SK_NACE']

        row_list.append(new_row)

DB_selected_partners = pd.DataFrame(row_list)
DB_selected_partners.to_csv('companies.csv', sep='|')